<a href="https://colab.research.google.com/github/sujithgunturu/colabtogit/blob/main/modelclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class LeafModel(tez.Model):
  def __init__(self, num_classes = 3):
    super().__init__()


    self.effnet = EfficientNet.from_pretrained("efficientnet-b4")
    self.dropout = nn.Dropout(0.1)
    self.out = nn.Linear(1792, num_classes)
    self.step_scheduler_after = "epoch"
 
  def loss(self, outputs, targets):
    if targets is None:
      return None
    return nn.CrossEntropyLoss()(outputs, targets)


  def monitor_metrics(self, outputs, targets):
    outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    acc =  metrics.accuracy_score(targets, outputs)
    return {
        "accuracy": acc
    }
  
  def fetch_optimizer(self):
    opt = torch.optim.Adam(self.parameters(), lr = 1e-3)
    return opt
  def fetch_scheduler(self):
    sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size = 0.7)
    return sch

  def forward(self, image, targets = None):  
    batch_size, _, _, _ = image.shape
    x = self.effnet.extract_features(image)
    x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
    outputs = self.out(self.dropout(x))
    if targets is not None:
       loss = self.loss(outputs, targets)
       metrics = self.monitor_metrics(outputs, targets)
       self.loss_tracker.append(loss)
       self.accuracy_tracker.append(metrics["accuracy"])

       return outputs, loss, metrics
    return outputs, None, None

In [ ]:
class LeafModel(tez.Model):
  def __init__(self, num_classes = 3):
    super().__init__()
    self.convnet = torchvision.models.resnet18(pretrained=True)
    self.convnet.fc = nn.Linear(512, num_classes)
    self.step_scheduler_after = "epoch"
   
  def loss(self, outputs, targets):
    if targets is None:
      return None
    return nn.CrossEntropyLoss()(outputs, targets)
    
  def monitor_metrics(self, outputs, targets):
    outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
    targets = targets.cpu().detach().numpy()
    acc =  metrics.accuracy_score(targets, outputs)
    return {
        "accuracy": acc
    }
  
  def fetch_optimizer(self):
    opt = torch.optim.Adam(self.parameters(), lr = 1e-3)
    return opt
  def fetch_scheduler(self):
    sch = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size = 0.7)
    return sch

  def forward(self, image, targets = None):
    outputs = self.convnet(image)
    if targets is not None:
       loss = self.loss(outputs, targets)
       metrics = self.monitor_metrics(outputs, targets)
       return outputs, loss, metrics
    return outputs, None, None